In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
path = '/content/gdrive/My Drive/'

In [28]:
# libraries

import pandas as pd
import numpy as np
from math import log

In [ ]:
# Task #1

def count_determinant(st1, st2):
  global matrices
  global keys

  number_row = []
  for nuc1, nuc2 in zip(st1, st2):
    if nuc1 == nuc2:
      number_row.append(2)
    else:
      number_row.append(1)

  matrix = [number_row]
  for _ in range(len(number_row) - 1):
    number_row = number_row[-1:] + number_row[:-1]
    matrix.append(number_row)

  det = np.linalg.det(matrix)
  matrices[det] = matrix
  keys.append(det)


seq1 = 'ATGAATTGGCAATGCA'
seq2 = 'ATAGGAATGCAATGAC'

matrices = dict()
keys = []

count_determinant(seq1, seq2)
for i in range(1, len(seq1) - 1):
  count_determinant(seq1[i:], seq2[:-i])

max_det = max(matrices)
corresponding_matrix = matrices[max_det]

corresponding_matrix

In [ ]:
# Task #2

def open_and_read(file):
  seqs = dict()

  with open(file, 'r') as fin:
      for line in fin:
        if '>' in line:
          ident = line[1:].rstrip()
        else:
          seqs[ident] = line.rstrip()

  return seqs


def count_gc_content(sequence):
  g_num = sequence.count('G')
  c_num = sequence.count('C')

  return (g_num + c_num) / len(sequence)


def create_naive_consensus(MSA):
  bases = ('A', 'T', 'G', 'C')

  num_seqs = len(MSA)
  seq_len = len(MSA[0])
  alignment = list(zip(*MSA))

  output = ''
  for pos in range(seq_len):
    pos_nucleotides = ''.join(list(alignment[pos]))
    frequencies = {base: pos_nucleotides.count(base) / num_seqs for base in bases}

    commonest_nucleotide = max(zip(frequencies.values(), frequencies.keys()))[1]
    output += commonest_nucleotide

  return output


def get_Kimura_dist(sequence, reference):
  L = len(reference)
  try:
    transitions = (('A', 'G'), ('G', 'A'), ('T', 'C'), ('C', 'T'))
    transversions = (('A', 'T'), ('A', 'C'), ('G', 'C'), ('G', 'T'), ('T', 'A'), ('T', 'G'), ('C', 'A'), ('C', 'G'))
    P = sum(1 for nuc_1, nuc_2 in zip(sequence, reference) if (nuc_1, nuc_2) in transitions) / L
    Q = sum(1 for nuc_1, nuc_2 in zip(sequence, reference) if (nuc_1, nuc_2) in transversions) / L
    return -1 / 2 * log(1 - 2 * P - Q) - 1 / 4 * log(1 - 2 * Q)
  except ValueError:
    return None


data = path + '2.sequences.var2.txt'
dict_form = open_and_read(data)

df = pd.DataFrame.from_dict(dict_form, orient='index')

gc_contents = []
for _, row in df.iterrows():
  gc_contents.append(count_gc_content(row[0]))

df['GC-content'] = gc_contents

consensus = create_naive_consensus(tuple(dict_form.values()))

kimura_dists = []
for _, row in df.iterrows():
  kimura_dists.append(get_Kimura_dist(row[0], consensus))

df['K2P'] = kimura_dists

df

In [ ]:
# Task #3

intervals = pd.read_csv(path + '3.var2.intervals.tsv', sep='\t')
variants = pd.read_csv(path + '3.var2.variants.tsv', sep='\t')

for index, row in intervals.iterrows():
  if row[1] > row[2]:
    intervals.loc[index, 'POSITION_START'], intervals.loc[index, 'POSITION_END'] = intervals.loc[index, 'POSITION_END'], intervals.loc[index, 'POSITION_START']

idx = pd.IntervalIndex.from_arrays(intervals['POSITION_START'], intervals['POSITION_END'], closed='both')

# very interesting but time's up